Cleaned and Merged Datasets: Hotel Bookings with Tourism Data based on Country
---
merged dataframe: hotel_tourism_df

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
#File to load
hotel_bookings_file = Path("Resources/hotel_bookings_dataset.csv")
tourism_file = Path("Resources/tourism_dataset.csv")

#Read file and store into Pandas DataFrames
hotel_data = pd.read_csv(hotel_bookings_file)
tourism_data = pd.read_csv(tourism_file)


In [3]:
# Remove specified columns from the DataFrame
columns_to_remove = [
    'arrival_date_week_number', 'arrival_date_day_of_month', 'meal',
    'market_segment', 'distribution_channel', 'previous_bookings_not_canceled',
    'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type',
    'agent', 'company', 'days_in_waiting_list', 'customer_type',
    'required_car_parking_spaces', 'total_of_special_requests',
    'reservation_status', 'reservation_status_date', 'is_canceled', 'lead_time',
    'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations'
]

hotel_data = hotel_data.drop(columns=columns_to_remove)
hotel_data.head()

,hotel,arrival_date_year,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,country,adr
0,Resort Hotel,2015,July,0,0,PRT,0.0
1,Resort Hotel,2015,July,0,0,PRT,0.0
2,Resort Hotel,2015,July,0,1,GBR,75.0
3,Resort Hotel,2015,July,0,1,GBR,75.0
4,Resort Hotel,2015,July,0,2,GBR,98.0


In [4]:
# Combine 'stays_in_weekend_nights' and 'stays_in_week_nights' into a new column 'total_stays_days'
hotel_data['total_stays_days'] = hotel_data['stays_in_weekend_nights'] + hotel_data['stays_in_week_nights']
hotel_data = hotel_data.drop(columns=['stays_in_weekend_nights', 'stays_in_week_nights'])

# Display the updated DataFrame
hotel_data

,hotel,arrival_date_year,arrival_date_month,country,adr,total_stays_days
0,Resort Hotel,2015,July,PRT,0.00,0
1,Resort Hotel,2015,July,PRT,0.00,0
2,Resort Hotel,2015,July,GBR,75.00,1
3,Resort Hotel,2015,July,GBR,75.00,1
4,Resort Hotel,2015,July,GBR,98.00,2
...,...,...,...,...,...,...
119385,City Hotel,2017,August,BEL,96.14,7
119386,City Hotel,2017,August,FRA,225.43,7
119387,City Hotel,2017,August,DEU,157.71,7
119388,City Hotel,2017,August,GBR,104.40,7


In [5]:
# Rearrange the columns
columns_order = [
    'country', 'adr','arrival_date_year', 'arrival_date_month', 'total_stays_days', 'hotel'
] + [col for col in hotel_data.columns if col not in ['country', 'adr', 'arrival_date_year', 'arrival_date_month', 'total_stays_days', 'hotel']]

hotel_data = hotel_data[columns_order]

# Rename columns
hotel_data = hotel_data.rename(columns={
    'country':'Country',
    'adr': 'Average daily rate',
    'hotel': 'Hotel type',
    'arrival_date_year': 'Arrival Year',
    'arrival_date_month': 'Arrival Month',
    'total_stays_days': 'Total Days of Stays'
})

hotel_data

,Country,Average daily rate,Arrival Year,Arrival Month,Total Days of Stays,Hotel type
0,PRT,0.00,2015,July,0,Resort Hotel
1,PRT,0.00,2015,July,0,Resort Hotel
2,GBR,75.00,2015,July,1,Resort Hotel
3,GBR,75.00,2015,July,1,Resort Hotel
4,GBR,98.00,2015,July,2,Resort Hotel
...,...,...,...,...,...,...
119385,BEL,96.14,2017,August,7,City Hotel
119386,FRA,225.43,2017,August,7,City Hotel
119387,DEU,157.71,2017,August,7,City Hotel
119388,GBR,104.40,2017,August,7,City Hotel


In [6]:
#convert abbreviation to country name
country_mapping = {
    'PRT': 'Portugal',
    'GBR': 'United Kingdom',
    'USA': 'United States',
    'ESP': 'Spain',
    'IRL': 'Ireland',
    'FRA': 'France',
    'ROU': 'Romania',
    'NOR': 'Norway',
    'OMN': 'Oman',
    'ARG': 'Argentina',
    'POL': 'Poland',
    'DEU': 'Germany',
    'BEL': 'Belgium',
    'CHE': 'Switzerland',
    'CN': 'China',
    'GRC': 'Greece',
    'ITA': 'Italy',
    'NLD': 'Netherlands',
    'DNK': 'Denmark',
    'RUS': 'Russia',
    'SWE': 'Sweden',
    'AUS': 'Australia',
    'EST': 'Estonia',
    'CZE': 'Czech Republic',
    'BRA': 'Brazil',
    'FIN': 'Finland',
    'MOZ': 'Mozambique',
    'BWA': 'Botswana',
    'LUX': 'Luxembourg',
    'SVN': 'Slovenia',
    'ALB': 'Albania',
    'IND': 'India',
    'CHN': 'China',
    'MEX': 'Mexico',
    'MAR': 'Morocco',
    'UKR': 'Ukraine',
    'SMR': 'San Marino',
    'LVA': 'Latvia',
    'PRI': 'Puerto Rico',
    'SRB': 'Serbia',
    'CHL': 'Chile',
    'AUT': 'Austria',
    'BLR': 'Belarus',
    'LTU': 'Lithuania',
    'TUR': 'Turkey',
    'ZAF': 'South Africa',
    'AGO': 'Angola',
    'ISR': 'Israel',
    'CYM': 'Cayman Islands',
    'ZMB': 'Zambia',
    'CPV': 'Cape Verde',
    'ZWE': 'Zimbabwe',
    'DZA': 'Algeria',
    'KOR': 'South Korea',
    'CRI': 'Costa Rica',
    'HUN': 'Hungary',
    'ARE': 'United Arab Emirates',
    'TUN': 'Tunisia',
    'JAM': 'Jamaica',
    'HRV': 'Croatia',
    'HKG': 'Hong Kong',
    'IRN': 'Iran',
    'GEO': 'Georgia',
    'AND': 'Andorra',
    'GIB': 'Gibraltar',
    'URY': 'Uruguay',
    'JEY': 'Jersey',
    'CAF': 'Central African Republic',
    'CYP': 'Cyprus',
    'COL': 'Colombia',
    'GGY': 'Guernsey',
    'KWT': 'Kuwait',
    'NGA': 'Nigeria',
    'MDV': 'Maldives',
    'VEN': 'Venezuela',
    'SVK': 'Slovakia',
    'FJI': 'Fiji',
    'KAZ': 'Kazakhstan',
    'PAK': 'Pakistan',
    'IDN': 'Indonesia',
    'LBN': 'Lebanon',
    'PHL': 'Philippines',
    'SEN': 'Senegal',
    'SYC': 'Seychelles',
    'AZE': 'Azerbaijan',
    'BHR': 'Bahrain',
    'NZL': 'New Zealand',
    'THA': 'Thailand',
    'DOM': 'Dominican Republic',
    'MKD': 'North Macedonia',
    'MYS': 'Malaysia',
    'ARM': 'Armenia',
    'JPN': 'Japan',
    'LKA': 'Sri Lanka',
    'CUB': 'Cuba',
    'CMR': 'Cameroon',
    'BIH': 'Bosnia and Herzegovina',
    'MUS': 'Mauritius',
    'COM': 'Comoros',
    'SUR': 'Suriname',
    'UGA': 'Uganda',
    'BGR': 'Bulgaria',
    'CIV': 'Ivory Coast',
    'JOR': 'Jordan',
    'SYR': 'Syria',
    'SGP': 'Singapore',
    'BDI': 'Burundi',
    'SAU': 'Saudi Arabia',
    'VNM': 'Vietnam',
    'PLW': 'Palau',
    'QAT': 'Qatar',
    'EGY': 'Egypt',
    'PER': 'Peru',
    'MLT': 'Malta',
    'MWI': 'Malawi',
    'ECU': 'Ecuador',
    'MDG': 'Madagascar',
    'ISL': 'Iceland',
    'UZB': 'Uzbekistan',
    'NPL': 'Nepal',
    'BHS': 'Bahamas',
    'MAC': 'Macau',
    'TGO': 'Togo',
    'TWN': 'Taiwan',
    'DJI': 'Djibouti',
    'STP': 'Sao Tome and Principe',
    'KNA': 'Saint Kitts and Nevis',
    'ETH': 'Ethiopia',
    'IRQ': 'Iraq',
    'HND': 'Honduras',
    'RWA': 'Rwanda',
    'KHM': 'Cambodia',
    'MCO': 'Monaco',
    'BGD': 'Bangladesh',
    'IMN': 'Isle of Man',
    'TJK': 'Tajikistan',
    'NIC': 'Nicaragua',
    'BEN': 'Benin',
    'VGB': 'British Virgin Islands',
    'TZA': 'Tanzania',
    'GAB': 'Gabon',
    'GHA': 'Ghana',
    'TMP': 'Timor-Leste',
    'GLP': 'Guadeloupe',
    'KEN': 'Kenya',
    'LIE': 'Liechtenstein',
    'GNB': 'Guinea-Bissau',
    'MNE': 'Montenegro',
    'UMI': 'United States Minor Outlying Islands',
    'MYT': 'Mayotte',
    'FRO': 'Faroe Islands',
    'MMR': 'Myanmar',
    'PAN': 'Panama',
    'BFA': 'Burkina Faso',
    'LBY': 'Libya',
    'MLI': 'Mali',
    'NAM': 'Namibia',
    'BOL': 'Bolivia',
    'PRY': 'Paraguay',
    'BRB': 'Barbados',
    'ABW': 'Aruba',
    'AIA': 'Anguilla',
    'SLV': 'El Salvador',
    'DMA': 'Dominica',
    'PYF': 'French Polynesia',
    'GUY': 'Guyana',
    'LCA': 'Saint Lucia',
    'ATA': 'Antarctica',
    'GTM': 'Guatemala',
    'ASM': 'American Samoa',
    'MRT': 'Mauritania',
    'NCL': 'New Caledonia',
    'KIR': 'Kiribati',
    'SDN': 'Sudan',
    'ATF': 'French Southern Territories',
    'SLE': 'Sierra Leone',
    'LAO': 'Laos',
}

# Replace initials with full country names
hotel_data['Country'] = hotel_data['Country'].replace(country_mapping)

In [7]:
#combine the data into single dataframe
hotel_tourism_df = pd.merge(hotel_data, tourism_data, how="left", on="Country")

#display data
hotel_tourism_df

,Country,Average daily rate,Arrival Year,Arrival Month,Total Days of Stays,Hotel type,Location,Category,Visitors,Rating,Revenue,Accommodation_Available
0,Portugal,0.00,2015,July,0,Resort Hotel,NaN,NaN,NaN,NaN,NaN,NaN
1,Portugal,0.00,2015,July,0,Resort Hotel,NaN,NaN,NaN,NaN,NaN,NaN
2,United Kingdom,75.00,2015,July,1,Resort Hotel,NaN,NaN,NaN,NaN,NaN,NaN
3,United Kingdom,75.00,2015,July,1,Resort Hotel,NaN,NaN,NaN,NaN,NaN,NaN
4,United Kingdom,98.00,2015,July,2,Resort Hotel,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13252697,France,225.43,2017,August,7,City Hotel,IfKotyaJFC,Nature,276317.0,3.53,325183.96,Yes
13252698,France,225.43,2017,August,7,City Hotel,gHXUrdticm,Cultural,40939.0,4.65,957026.85,Yes
13252699,Germany,157.71,2017,August,7,City Hotel,NaN,NaN,NaN,NaN,NaN,NaN
13252700,United Kingdom,104.40,2017,August,7,City Hotel,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Remove specified columns from the DataFrame
hotel_tourism_df_clean = [
    'Location', 'Category', 'Accommodation_Available'
]

hotel_tourism_df = hotel_tourism_df.drop(columns=hotel_tourism_df_clean)
hotel_tourism_df.head()

,Country,Average daily rate,Arrival Year,Arrival Month,Total Days of Stays,Hotel type,Visitors,Rating,Revenue
0,Portugal,0.0,2015,July,0,Resort Hotel,NaN,NaN,NaN
1,Portugal,0.0,2015,July,0,Resort Hotel,NaN,NaN,NaN
2,United Kingdom,75.0,2015,July,1,Resort Hotel,NaN,NaN,NaN
3,United Kingdom,75.0,2015,July,1,Resort Hotel,NaN,NaN,NaN
4,United Kingdom,98.0,2015,July,2,Resort Hotel,NaN,NaN,NaN


In [9]:
#display countries from the hotel_data dataframe
countries_total = hotel_data["Country"].nunique()
countries_total

176

In [10]:
#display all countries from the hotel_data dataframe
all_countries = hotel_data["Country"].unique()
all_countries

array(['Portugal', 'United Kingdom', 'United States', 'Spain', 'Ireland',
       'France', nan, 'Romania', 'Norway', 'Oman', 'Argentina', 'Poland',
       'Germany', 'Belgium', 'Switzerland', 'China', 'Greece', 'Italy',
       'Netherlands', 'Denmark', 'Russia', 'Sweden', 'Australia',
       'Estonia', 'Czech Republic', 'Brazil', 'Finland', 'Mozambique',
       'Botswana', 'Luxembourg', 'Slovenia', 'Albania', 'India', 'Mexico',
       'Morocco', 'Ukraine', 'San Marino', 'Latvia', 'Puerto Rico',
       'Serbia', 'Chile', 'Austria', 'Belarus', 'Lithuania', 'Turkey',
       'South Africa', 'Angola', 'Israel', 'Cayman Islands', 'Zambia',
       'Cape Verde', 'Zimbabwe', 'Algeria', 'South Korea', 'Costa Rica',
       'Hungary', 'United Arab Emirates', 'Tunisia', 'Jamaica', 'Croatia',
       'Hong Kong', 'Iran', 'Georgia', 'Andorra', 'Gibraltar', 'Uruguay',
       'Jersey', 'Central African Republic', 'Cyprus', 'Colombia',
       'Guernsey', 'Kuwait', 'Nigeria', 'Maldives', 'Venezuela',
    

In [11]:
#display merged dataset
hotel_tourism_df

,Country,Average daily rate,Arrival Year,Arrival Month,Total Days of Stays,Hotel type,Visitors,Rating,Revenue
0,Portugal,0.00,2015,July,0,Resort Hotel,NaN,NaN,NaN
1,Portugal,0.00,2015,July,0,Resort Hotel,NaN,NaN,NaN
2,United Kingdom,75.00,2015,July,1,Resort Hotel,NaN,NaN,NaN
3,United Kingdom,75.00,2015,July,1,Resort Hotel,NaN,NaN,NaN
4,United Kingdom,98.00,2015,July,2,Resort Hotel,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
13252697,France,225.43,2017,August,7,City Hotel,276317.0,3.53,325183.96
13252698,France,225.43,2017,August,7,City Hotel,40939.0,4.65,957026.85
13252699,Germany,157.71,2017,August,7,City Hotel,NaN,NaN,NaN
13252700,United Kingdom,104.40,2017,August,7,City Hotel,NaN,NaN,NaN
